In [447]:
import numpy as np

In [448]:
class ANN:
    #List of List for each layer along with their information
    #Shape: [ [layerType,activationF(),neuronCount],[layerType,activationF(),neuronCount]]
    layer_info = None
    #List of weight matrices of each layer
    #Shape: dxk (where d is no of input)(k is no of output)
    w_l_ij= None
    #List of bias vector of each layer
    #Shape: 1xk (where k is no of output/neuron)
    b_l_j = None
    #List of output of each neuron
    #Shape: 1xd (where d is dimension)
    a_l_ij = None
    #List of d(a_l+1)/da_l (where a_l+1 is output of layer)(a_l is input of layer)
    #Shape: 1xd (same shape as input of layer)
    delta_l_ij = None
    #List of matrices of dl/dw for each layer
    dl_db = None
    dl_dw = None
    #Dictionary for differentiation function for a input function. diff[f()] => df()
    diff = {}
    input_featureCount = None
    input_count = None
    def __init__(self,xShape,rseed = 10,listLayer= None):
        np.random.seed(rseed)
        inputCount,inputDim = xShape
        self.input_featureCount = inputDim
        self.input_count = inputCount
        self.layer_info = [['None','None',self.input_featureCount]]
        self.w_l_ij = ['None']
        self.b_l_j = ['None']
        self.optimizer = None
        self.diff[ANN.activation_linear] = ANN.diff_activation_linear
        self.diff[ANN.activation_sigmoid] = ANN.diff_activation_sigmoid
        self.diff[ANN.activation_tanh] = ANN.diff_activation_tanh

    def compile(self,optimizer,lossFunction):
        self.optimizer = optimizer
        self.layer_info.append(['Loss',lossFunction])

    def __str__(self):
        return

    def addLayers_Dense(self,neuronCount,activationFunction):
            self.layer_info.append(['Dense',activationFunction,neuronCount])
            w = np.random.rand(self.layer_info[-2][2],neuronCount)
            b = np.random.rand(1,neuronCount)
            self.w_l_ij.append(w)
            self.b_l_j.append(b)

    def addLayers_softmax(self):
        self.layer_info.append(['Softmax',self.layer_info[-1][2]])
        self.w_l_ij.append('None')
        self.b_l_j.append('None')

    def forward_layer_dense(self,layerNo):
        if layerNo < 1:
            print("ERRRROR")
        #Output = activationFunction(a_l-1 @ w_l + b_l)
        output = self.layer_info[layerNo][1](self.a_l_ij[layerNo-1] @ self.w_l_ij[layerNo] + self.b_l_j[layerNo]) 
        self.a_l_ij.append(output)
    
    def backward_layer_dense(self,layerNo):
        #at each layer we need to calculate da_l/da_l-1, and da_l/dw_l
        #Shape: dxk
        da_1da = self.diff[self.layer_info[layerNo][1]](self.a_l_ij[layerNo]).T @ self.w_l_ij[layerNo].T
        #Shape: dxk
        da_l1_dw_l = np.zeros(self.w_l_ij[layerNo].shape)
        for i in range(da_l1_dw_l.shape[0]):
            for j in range(da_l1_dw_l.shape[1]):
                da_l1_dw_l[i][j] = self.diff[self.layer_info[layerNo][1]](self.a_l_ij[layerNo][0][j],'SINGLE') * self.a_l_ij[layerNo-1][0][i]

        #multiply both by dL/da_l to get dL/da_l-1 and dL/dw_l
        #print(layerNo)
        # print(da_1da)
        # print()
        self.delta_l_ij[layerNo] = self.delta_l_ij[layerNo+1] @ da_1da

        #print("AAA:",da_l1_dw_l,"\n", self.delta_l_ij[layerNo+1])
        temp = np.zeros(self.w_l_ij[layerNo].shape)
        for i in range(temp.shape[0]):
            for j in range(temp.shape[1]):
                temp[i][j] = da_l1_dw_l[i][j] * self.delta_l_ij[layerNo+1][0][j]

        self.dl_dw.append(temp)
        self.dl_db.append(self.delta_l_ij[layerNo+1])
    
    def forward_layer_softmax(self,layerNo):
        if layerNo < 1: 
            print("ERRORRR")
        denom = np.sum( np.exp(self.a_l_ij[layerNo-1]) )
        output = np.exp(self.a_l_ij[layerNo-1]) / denom
        self.a_l_ij.append(output)

    def backward_layer_softmax(self,layerNo):
        pass


    def forward_layer_loss(self,layerNo,y):
        y_hat  =  self.a_l_ij[-1]
        lossFunction = self.layer_info[layerNo][1]
        return lossFunction(y_hat,y)

    def forwardProp(self,x,y):
        self.a_l_ij = []
        self.a_l_ij.append(x)
        for layerNo in range(len(self.layer_info)):
            if self.layer_info[layerNo][0] == 'Dense':
                self.forward_layer_dense(layerNo)
            if self.layer_info[layerNo][0] == 'Softmax':
                self.forward_layer_softmax(layerNo)
            if self.layer_info[layerNo][0] == 'Loss':
                cost = self.forward_layer_loss(layerNo,y)
                return cost
            
    #Handle softmax backpropogation
    def backwardProp(self,y):
        self.dl_dw = []
        self.dl_db = []
        #-1 for getting index, -1 for loss layer
        lastIndex = len(self.layer_info) - 2
        self.delta_l_ij = [i for i in range(0,lastIndex+1)]
        
        #MSE LOSS And Dense
        if(self.layer_info[-1][1] == ANN.loss_MSE):
            self.delta_l_ij[lastIndex] = 2 * ( (self.a_l_ij[lastIndex] - y) @ self.w_l_ij[lastIndex].T) 
            self.dl_dw.append(2*(self.a_l_ij[lastIndex-1].T @ self.a_l_ij[lastIndex] - y))
            self.dl_db.append(2*(self.a_l_ij[lastIndex] - y))
        elif(self.layer_info[-1][1] == ANN.loss_crossEntropy):
            self.dl_dw.append(['None'])
            self.delta_l_ij[lastIndex] = self.a_l_ij[lastIndex] - y 
             
        #BackPropogation
        for layerNo in range(lastIndex-1,0,-1):
            if self.layer_info[layerNo][0] == 'Dense':
                self.backward_layer_dense(layerNo)
                
        self.dl_dw.append(['None'])
        self.dl_db.append(['None'])
        self.dl_dw.reverse()
        self.dl_db.reverse()


            
    def fit(self,x,y,eta):
        #Hyperparameters if required: 
        epoch = 1000
        tolerance = 1e-5
        self.optimizer(self,x,y,eta,epoch,tolerance)

    @staticmethod    
    def optimizer_gradientDescent(Obj,x,y,eta,epoch,tolerance):
        for i in  range(epoch):
            errorSum = 0
            print("Epoch:",i,end="\t")
            for j in range(1):
                #x and y both are 2d matrix
                x_1xd = x[j:j+1]
                y_1xk = y[j:j+1]
                errorSum += Obj.forwardProp(x_1xd,y_1xk)
                Obj.backwardProp(y_1xk)
                for index in range(1,len(Obj.w_l_ij)):
                    if( 'None' in Obj.dl_dw[index]):
                        continue
                    Obj.w_l_ij[index] = Obj.w_l_ij[index] - (eta * Obj.dl_dw[index])
                    Obj.b_l_j[index] = Obj.b_l_j[index] - (eta * Obj.dl_db[index] )
            print("Error:",errorSum)
            if(errorSum < tolerance):
                return

    @staticmethod
    def loss_MSE(P,y):
        return np.sum((P - y)**2)   

    @staticmethod
    def loss_crossEntropy(P,y):
        loss = np.log(P ** y)
        return -np.sum(loss)
      
    @staticmethod
    def activation_linear(z):
        return z
    
    @staticmethod
    def diff_activation_linear(x,flg='None'):
        if(flg != 'SINGLE'):
            return np.identity(x.shape[1])
        else:
            return x
    
    @staticmethod
    def activation_sigmoid(z):    
        return 1 / (1 + np.exp(-z))
    
    @staticmethod    
    def diff_activation_sigmoid(a,flg='None'):
        if(flg !='SINGLE'):
            x = a*(1-a)
            return np.diag(x[0])
        else:
            return a * (1- a)

    @staticmethod
    def activation_tanh(z):
        sinh = np.exp(z) - np.exp(-z)
        cosh = np.exp(z) + np.exp(-z)
        return sinh / cosh
    
    @staticmethod
    def diff_activation_tanh(z,flg='None'):
        if(flg != 'SINGLE'):
            x = 1 - (z*z)
            return np.diag(x[0])
        else:
            return 1 - (z * z)


In [449]:
import pandas as pd
import numpy as np
import sklearn.datasets as sk

In [450]:
dictt = sk.fetch_california_housing()
x = dictt.data
y = np.array(dictt.target)
y = y.T
x

array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]])

In [451]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer().fit(x)
x = transformer.transform(x)

### Q2 (a)

In [452]:
model = ANN(x.shape,42)
model.addLayers_Dense(1,ANN.activation_linear)
model.compile(ANN.optimizer_gradientDescent,ANN.loss_MSE)
model.fit(x,y,1e-2)

Epoch: 0	Error: 15.522170015157613
Epoch: 1	Error: 14.406532173851268
Epoch: 2	Error: 13.374658586470662
Epoch: 3	Error: 12.420130205621323
Epoch: 4	Error: 11.537025314419967
Epoch: 5	Error: 10.719880772694628
Epoch: 6	Error: 9.96365629200614
Epoch: 7	Error: 9.263701502410605
Epoch: 8	Error: 8.615725592454046
Epoch: 9	Error: 8.015769321007385
Epoch: 10	Error: 7.460179215324669
Epoch: 11	Error: 6.945583784246708
Epoch: 12	Error: 6.46887158887172
Epoch: 13	Error: 6.02717102536433
Epoch: 14	Error: 5.617831685956112
Epoch: 15	Error: 5.238407174680884
Epoch: 16	Error: 4.886639264056085
Epoch: 17	Error: 4.5604432878320615
Epoch: 18	Error: 4.257894673143483
Epoch: 19	Error: 3.977216522965985
Epoch: 20	Error: 3.7167681667571295
Epoch: 21	Error: 3.4750346035899766
Epoch: 22	Error: 3.250616768013465
Epoch: 23	Error: 3.0422225543349537
Epoch: 24	Error: 2.848658540054084
Epoch: 25	Error: 2.668822353816276
Epoch: 26	Error: 2.5016956375300157
Epoch: 27	Error: 2.346337556233079
Epoch: 28	Error: 2.201

### Q2 (b)

In [453]:
model = ANN(x.shape,42)
model.addLayers_Dense(13,ANN.activation_sigmoid)
model.addLayers_Dense(1,ANN.activation_linear)
model.compile(ANN.optimizer_gradientDescent,ANN.loss_MSE)
model.fit(x,y,1e-2)

Epoch: 0	Error: 0.40154490017651384
Epoch: 1	Error: 0.5667323622332192
Epoch: 2	Error: 0.7244884039271857
Epoch: 3	Error: 0.869560000038593
Epoch: 4	Error: 0.9998844539501808
Epoch: 5	Error: 1.1152731320462685
Epoch: 6	Error: 1.216561673574752
Epoch: 7	Error: 1.3050768089941243
Epoch: 8	Error: 1.3823133625471267
Epoch: 9	Error: 1.4497477523235367
Epoch: 10	Error: 1.5087380826062093
Epoch: 11	Error: 1.560477704204257
Epoch: 12	Error: 1.605980681884925
Epoch: 13	Error: 1.646085417319428
Epoch: 14	Error: 1.6814678561272087
Epoch: 15	Error: 1.7126590856431692
Epoch: 16	Error: 1.740064294479876
Epoch: 17	Error: 1.7639814244392205
Epoch: 18	Error: 1.7846186781149573
Epoch: 19	Error: 1.8021105381892941
Epoch: 20	Error: 1.8165322304860534
Epoch: 21	Error: 1.8279127032865377
Epoch: 22	Error: 1.836246252985709
Epoch: 23	Error: 1.8415029351123833
Epoch: 24	Error: 1.8436378823145296
Epoch: 25	Error: 1.8425996216258451
Epoch: 26	Error: 1.8383374516203317
Epoch: 27	Error: 1.8308079119353688
Epoch: 2

### Q2 (c)

In [454]:
model = ANN(x.shape,42)
model.addLayers_Dense(13,ANN.activation_sigmoid)
model.addLayers_Dense(13,ANN.activation_sigmoid)
model.addLayers_Dense(1,ANN.activation_linear)
model.compile(ANN.optimizer_gradientDescent,ANN.loss_MSE)
model.fit(x,y,1e-1)

Epoch: 0	Error: 2.044462352357814
Epoch: 1	Error: 5.9319187712795465
Epoch: 2	Error: 19.2262157017506
Epoch: 3	Error: 57.93921554494745
Epoch: 4	Error: 173.82339100615695
Epoch: 5	Error: 178.2605517155229
Epoch: 6	Error: 13.081426708375425
Epoch: 7	Error: 8.28960873781668
Epoch: 8	Error: 5.070241401981688
Epoch: 9	Error: 2.5172094727582865
Epoch: 10	Error: 0.28032774344828987
Epoch: 11	Error: 0.20769917321480894
Epoch: 12	Error: 0.01451434109920243
Epoch: 13	Error: 0.1255513961419844
Epoch: 14	Error: 0.07359676657653302
Epoch: 15	Error: 0.44843439278510344
Epoch: 16	Error: 0.7776111514267396
Epoch: 17	Error: 8.18216311042202
Epoch: 18	Error: 4.194560464322516
Epoch: 19	Error: 0.07993642487948123
Epoch: 20	Error: 1.6282097601837129
Epoch: 21	Error: 3.0127640841780057
Epoch: 22	Error: 138.82056112407338
Epoch: 23	Error: 14.896296126464218
Epoch: 24	Error: 9.533629510171394
Epoch: 25	Error: 6.101318607298842
Epoch: 26	Error: 3.1235114317180974
Epoch: 27	Error: 6718.043283531555
Epoch: 28	

C:\Users\Prakhar\AppData\Local\Temp\ipykernel_8452\1723139635.py:193: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Error: 76.01222116187799
Epoch: 377	Error: 48.64782154360189
Epoch: 378	Error: 31.134605787905215
Epoch: 379	Error: 19.92614770425934
Epoch: 380	Error: 12.752734530725975
Epoch: 381	Error: 8.161750099664625
Epoch: 382	Error: 5.223520063785361
Epoch: 383	Error: 3.3430528408226317
Epoch: 384	Error: 2.139553818126484
Epoch: 385	Error: 1.3693144436008713
Epoch: 386	Error: 73385.12443492029
Epoch: 387	Error: 3037.720486061331
Epoch: 388	Error: 1944.1411110792524
Epoch: 389	Error: 1244.2503110907212
Epoch: 390	Error: 796.3201990980616
Epoch: 391	Error: 509.64492742275945
Epoch: 392	Error: 326.172753550566
Epoch: 393	Error: 208.7505622723622
Epoch: 394	Error: 133.60035985431182
Epoch: 395	Error: 85.50423030675955
Epoch: 396	Error: 54.72270739632611
Epoch: 397	Error: 35.02253273364871
Epoch: 398	Error: 22.414420949535177
Epoch: 399	Error: 14.345229407702508
Epoch: 400	Error: 9.180946820929606
Epoch: 401	Error: 5.875805965394947
Epoch: 402	Error: 3.7605158178527653
Epoch: 403	Error: 2.406730123

### Q3

In [455]:
dictt = sk.load_digits()
x = dictt.data
y = dictt.target
x

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])

In [456]:
def oneHot_encoding(categories ,labels):
    num_samples = len(labels)
    num_categories = len(categories)
    one_hot = np.zeros((num_samples, num_categories))

    # Perform one-hot encoding
    for i, label in enumerate(labels):
        index = categories.index(label)
        one_hot[i, index] = 1
    return one_hot

In [457]:
y_encoded = oneHot_encoding([0,1,2,3,4,5,6,7,8,9],y)
y_encoded.shape

(1797, 10)

### Q3 (a)

In [458]:
model = ANN(x.shape,42)
model.addLayers_Dense(89, ANN.activation_tanh)
model.addLayers_Dense(10, ANN.activation_sigmoid)
model.compile(ANN.optimizer_gradientDescent, ANN.loss_MSE)
model.fit(x, y_encoded, 1e-1)

Epoch: 0	Error: 9.0
Epoch: 1	Error: 8.999999998481723
Epoch: 2	Error: 8.905705044696033
Epoch: 3	Error: 1.935758350701746e-06


### Q3 (b)

In [459]:
model = ANN(x.shape,42)
model.addLayers_Dense(89,ANN.activation_tanh)
model.addLayers_Dense(10,ANN.activation_linear)
model.addLayers_softmax()
model.compile(ANN.optimizer_gradientDescent,ANN.loss_crossEntropy)
model.fit(x,y_encoded,1e-3)

Epoch: 0	Error: 3.0386562807490933
Epoch: 1	Error: 0.18181534479237224
Epoch: 2	Error: 0.08281241685643811
Epoch: 3	

Error: 0.05602835052205229
Epoch: 4	Error: 0.04269291301835143
Epoch: 5	Error: 0.03459500844295964
Epoch: 6	Error: 0.029123931544781895
Epoch: 7	Error: 0.025168238688433712
Epoch: 8	Error: 0.02216979154342514
Epoch: 9	Error: 0.01981609245724341
Epoch: 10	Error: 0.01791800749402758
Epoch: 11	Error: 0.016354134384501444
Epoch: 12	Error: 0.015042869624187844
Epoch: 13	Error: 0.013927282437154756
Epoch: 14	Error: 0.012966417227980566
Epoch: 15	Error: 0.012130041621516664
Epoch: 16	Error: 0.011395343231987907
Epoch: 17	Error: 0.01074477919408114
Epoch: 18	Error: 0.010164634112577293
Epoch: 19	Error: 0.009644027891663636
Epoch: 20	Error: 0.00917421754087057
Epoch: 21	Error: 0.00874809596740863
Epoch: 22	Error: 0.008359825734882958
Epoch: 23	Error: 0.008004567150620272
Epoch: 24	Error: 0.007678273467010534
Epoch: 25	Error: 0.007377534610257968
Epoch: 26	Error: 0.007099456515012168
Epoch: 27	Error: 0.0068415669354694765
Epoch: 28	Error: 0.006601741186810721
Epoch: 29	Error: 0.00637814305921593